In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
from datetime import timedelta

In [2]:
import os

if (os.getcwd() != "../data/raw") :
    os.chdir("../data/raw")
os.getcwd()

'/root/restaurants/data/raw'

In [3]:
visit_data = pd.read_csv("air_visit_data.csv")
visit_data["visit_date"] = pd.to_datetime(visit_data["visit_date"])
visit_data = visit_data.rename(columns={"air_store_id": "store_id"})
visit_data["visit_date"] = visit_data["visit_date"].dt.date
visit_data

,store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6
...,...,...,...
252103,air_24e8414b9b07decb,2017-04-18,6
252104,air_24e8414b9b07decb,2017-04-19,6
252105,air_24e8414b9b07decb,2017-04-20,7
252106,air_24e8414b9b07decb,2017-04-21,8


In [4]:
store_info = pd.read_csv("air_store_info.csv")
store_info = store_info.rename(
    columns={
        "air_store_id": "store_id",
        "air_genre_name": "genre_name",
        "air_area_name": "area_name",
    }
)

In [5]:
date_info = pd.read_csv("date_info.csv")
visit_data = visit_data.rename(columns={"visit_date": "date"})
visit_data["date"] = visit_data["date"].astype("string")

date_info = date_info.rename(columns={"calendar_date": "date"})
date_info["date"] = date_info["date"].astype("string")

In [28]:
data = pd.merge(visit_data, date_info, on="date", how="left")
data = pd.merge(data, store_info, on="store_id", how="left")
data["date"] = pd.to_datetime(data["date"])
data["year"] = data["date"].dt.year
data["month"] = data["date"].dt.month
data["day"] = data["date"].dt.day
data = data.sort_values(["store_id", "date"]).reset_index(drop=True)
data

,store_id,date,visitors,day_of_week,holiday_flg,genre_name,area_name,latitude,longitude,year,month,day
0,air_00a91d42b08b08d9,2016-07-01,35,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,1
1,air_00a91d42b08b08d9,2016-07-02,9,Saturday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,2
2,air_00a91d42b08b08d9,2016-07-04,20,Monday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,4
3,air_00a91d42b08b08d9,2016-07-05,25,Tuesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,5
4,air_00a91d42b08b08d9,2016-07-06,29,Wednesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,6
...,...,...,...,...,...,...,...,...,...,...,...,...
252103,air_fff68b929994bfbd,2017-04-18,6,Tuesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,2017,4,18
252104,air_fff68b929994bfbd,2017-04-19,2,Wednesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,2017,4,19
252105,air_fff68b929994bfbd,2017-04-20,2,Thursday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,2017,4,20
252106,air_fff68b929994bfbd,2017-04-21,4,Friday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,2017,4,21


In [32]:
def add_lag(df, lag, column):
    lag_data = []

    for store_id, store_data in df.groupby("store_id"):
        for temp in store_data.groupby(column):
            temp_rolled = temp[["date", "visitors"]].rolling(f"{lag}D", on="date", min_periods=1).mean()
            temp_rolled["store_id"] = store_id
            temp_rolled[f"lag_{column}_{lag}"] = temp_rolled["visitors"]
            temp_rolled = temp_rolled.drop(columns="visitors")

            lag_data.append(temp_rolled)
    
    lag_data = pd.concat(lag_data)
    
    return df.merge(lag_data, on=["store_id", "date"], how="left")


In [34]:
new_data = deepcopy(data)

In [35]:
for lag in [21, 35]:
    new_data = add_lag(data, lag, "day_of_week")
    new_data = add_lag(new_data, lag, "holiday_flg")


In [ ]:
new_data["store_id"].unique()

In [10]:
data

,store_id,date,visitors,day_of_week,holiday_flg,genre_name,area_name,latitude,longitude,year,month,day,lagday_of_week21
0,air_00a91d42b08b08d9,2016-07-01,35,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,1,35.0
1,air_00a91d42b08b08d9,2016-07-02,9,Saturday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,2,NaN
2,air_00a91d42b08b08d9,2016-07-04,20,Monday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,4,NaN
3,air_00a91d42b08b08d9,2016-07-05,25,Tuesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,5,NaN
4,air_00a91d42b08b08d9,2016-07-06,29,Wednesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,7,6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
252103,air_fff68b929994bfbd,2017-04-18,6,Tuesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,2017,4,18,NaN
252104,air_fff68b929994bfbd,2017-04-19,2,Wednesday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,2017,4,19,NaN
252105,air_fff68b929994bfbd,2017-04-20,2,Thursday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,2017,4,20,NaN
252106,air_fff68b929994bfbd,2017-04-21,4,Friday,0,Bar/Cocktail,Tōkyō-to Nakano-ku Nakano,35.708146,139.666288,2017,4,21,6.0


In [9]:
# from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_relevant_features
df= data[["store_id", "date", "day_of_week", "holiday_flg", "genre_name", "area_name", "latitude", "longitude", "year", "month", "day"]]

from sklearn.preprocessing import LabelEncoder
df[["day_of_week", "genre_name", "area_name"]] = df[["day_of_week", "genre_name", "area_name"]].apply(LabelEncoder().fit_transform)
y = data[["store_id", "visitors"]].set_index("store_id").squeeze()
X = extract_relevant_features(df, y, column_id='store_id', column_sort="date")
X

/tmp/ipykernel_124483/4026643752.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["day_of_week", "genre_name", "area_name"]] = df[["day_of_week", "genre_name", "area_name"]].apply(LabelEncoder().fit_transform)
Feature Extraction:  95%|█████████▌| 19/20 [14:52<00:34, 34.73s/it]